In [97]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pyprind
import psycopg2 as pg
import pandas.io.sql as pd_sql
import scipy

%matplotlib inline


In [73]:
df_users = pd.read_csv('data/11-5 user factor.csv', index_col=0)
df_items = pd.read_csv('data/11-5 book factor.csv', index_col=0)

In [74]:
df_users.rename({'0':'userid'}, inplace=True, axis=1)
df_items.rename({'0':'hash'}, inplace=True, axis=1)

In [75]:
print(df_users.head(1))
print(df_items.head(1))

     userid        1         2         3         4         5         6  \
0  16668577 -0.17293  0.280011  0.439758  0.660478 -0.309844  0.656198   

          7         8         9    ...           32        33        34  \
0  0.344646  1.102115  0.202402    ...     0.555684  0.066521  0.044912   

         35       36        37       38        39        40        41  
0 -0.175854  0.01597  0.233702 -0.26174 -0.559784  0.316549 -0.491456  

[1 rows x 42 columns]
                                    hash        1         2        3  \
0  hash_66bd251f2d18ce183d18bd387d2e973d -0.17293 -0.028103 -0.22346   

          4         5         6         7         8        9    ...     \
0 -0.035469  0.113486  0.049002  0.120683 -0.214678 -0.10782    ...      

         32        33        34        35        36        37        38  \
0  0.013351 -0.036024  0.085505 -0.184906 -0.011864 -0.118992 -0.021779   

         39        40        41  
0 -0.172343 -0.062204  0.165561  

[1 rows x 42 column

In [76]:
test_book = np.array(df_items.loc[100])
bi = test_book[1]
qi = test_book[2:]
ratings = []
t0 = time.time()
for row in df_users.itertuples():
    bu = row[2]
    pu = np.array(row[3:])
    rating = bi+ bu + np.dot(qi, pu)
    ratings.append(rating)
ratings = np.array(ratings)
t1 = time.time()
print(t1-t0)

0.09433221817016602


In [98]:
pu = np.array(df_users.iloc[:,2:])
pu = pu.astype(float)

bu = np.array(df_users.iloc[:, 1])

means = []
stdevs = []
normals = []

progbar = pyprind.ProgBar(len(df_items))

for test_book in df_items.itertuples():
    bi = test_book[2]
    qi = test_book[3:]
    ratings = []
    factors = qi @ pu.T
    ratings = bi+bu+factors
    means.append(np.mean(factors))
    stdevs.append(np.std(factors))
    normals.append(scipy.stats.normaltest(factors))
    progbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:17


In [78]:
test_book = df_items.iloc[0]
qi = np.array(test_book[2:])
qi = qi.astype(float)


In [103]:
normals_tf = [x[1]<.001 for x in normals]

sum(normals_tf)
len(normals_tf)

31237

In [79]:
df_items.head()

,hash,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,hash_66bd251f2d18ce183d18bd387d2e973d,-0.172930,-0.028103,-0.223460,-0.035469,0.113486,0.049002,0.120683,-0.214678,-0.107820,...,0.013351,-0.036024,0.085505,-0.184906,-0.011864,-0.118992,-0.021779,-0.172343,-0.062204,0.165561
1,hash_8466618ac1ae058828011a26e7b5e60f,0.208797,-0.024963,0.046442,-0.656345,-0.158792,-0.083407,0.035388,0.587255,0.134119,...,0.759217,-0.737455,-0.100480,0.348089,0.324024,0.216771,-0.451586,0.245189,0.100525,-0.454094
2,hash_87ca483b2e90f93980ee25df09eacf99,0.638755,-0.256699,0.306707,-0.057934,0.466045,0.164737,-0.191644,0.289071,-0.661782,...,-0.574859,-0.269398,0.292238,0.178438,-0.153396,-0.169998,-0.162705,0.238297,-0.275924,-0.279752
3,hash_2032bf9577b6c4791aa1e00926f7aafd,0.813292,-0.427983,0.464713,-0.430439,0.049712,-0.403589,0.494130,0.576196,-0.027051,...,-0.048657,0.074584,-0.075561,0.226364,-0.285634,-0.236753,0.191464,0.025262,-0.402932,-0.032671
4,hash_f33c29ef8819e13f557be54a10d2e6c3,0.023426,-0.180042,0.322995,-0.159424,-0.331475,0.173746,-0.114023,0.307676,0.226467,...,-0.075873,-0.106311,0.024278,-0.221217,-0.007434,-0.065817,-0.560487,-0.271060,0.227242,-0.076418


In [80]:
df_items.hash

0        hash_66bd251f2d18ce183d18bd387d2e973d
1        hash_8466618ac1ae058828011a26e7b5e60f
2        hash_87ca483b2e90f93980ee25df09eacf99
3        hash_2032bf9577b6c4791aa1e00926f7aafd
4        hash_f33c29ef8819e13f557be54a10d2e6c3
5        hash_c161958337793b9ee8a934a624fd5c01
6        hash_1c98efb8091b2f2dce008a6cbd399f02
7        hash_9ebd41b6891f7f851db6dc85b43a70d3
8        hash_31530376e6d20a27be3101583c663b14
9        hash_3165929abac6bd3f2fa08e67bff1d10e
10       hash_9d1c2309f63a885ad2b8743855a5747c
11       hash_56e19a67826fb804bb3fab92d9784745
12       hash_eb6a191f81aa975c55317b624101380e
13       hash_4f496e7d4f06e2327fb90b02254d83ad
14       hash_fe862c76dc32e5261d019be948205e5f
15       hash_bd29e8131276eaf11951a6f5f31811bd
16       hash_5cb4c17b11ed2ac1449f0adb94feb365
17       hash_c01ae68e2b856d9e85a6b412cefcf012
18       hash_405c558e6f6de91918d2588043a27a8c
19       hash_ec9ef0ea169a09c34c90e123ac6ce9e1
20       hash_cb6aedd10187484abebe2b699eeba8f5
21       hash

In [83]:
df = pd.DataFrame({'hash': df_items['hash'],
                   'mu':means,
                   'stdev':stdevs,
                   'normal':normals_tf
                  'factor1':df_items['2'],
                  'factor2':df_items['3'],
                  'factor3':df_items['4'],
                  'factor4':df_items['5'],
                  'factor5':df_items['6']})


In [84]:
df.head()

,hash,mu,stdev,factor1,factor2,factor3,factor4,factor5
0,hash_66bd251f2d18ce183d18bd387d2e973d,-0.001731,0.212350,-0.028103,-0.223460,-0.035469,0.113486,0.049002
1,hash_8466618ac1ae058828011a26e7b5e60f,-0.000184,0.664273,-0.024963,0.046442,-0.656345,-0.158792,-0.083407
2,hash_87ca483b2e90f93980ee25df09eacf99,0.005866,0.673671,-0.256699,0.306707,-0.057934,0.466045,0.164737
3,hash_2032bf9577b6c4791aa1e00926f7aafd,0.006618,0.577222,-0.427983,0.464713,-0.430439,0.049712,-0.403589
4,hash_f33c29ef8819e13f557be54a10d2e6c3,-0.005472,0.400177,-0.180042,0.322995,-0.159424,-0.331475,0.173746


In [85]:
book_info = pd.read_csv('data/11-5 full cleaned df.csv')

In [86]:
book_info.drop_duplicates('bookid', inplace=True)
book_info_sub = book_info[['hash', 'times_rated', 'avg_rating', 'bookid']]
book_info_sub.set_index('hash', inplace=True)

In [87]:
df2 = df.join(book_info_sub, on='hash')

df2['title'] = df2.bookid.apply(lambda x: x.split(" || ")[1])
df2['author'] = df2.bookid.apply(lambda x: x.split(" || ")[0])
df2['log_times_rated'] = np.log10(df2.times_rated+1)
df2.head()

,hash,mu,stdev,factor1,factor2,factor3,factor4,factor5,times_rated,avg_rating,bookid,title,author,log_times_rated
0,hash_66bd251f2d18ce183d18bd387d2e973d,-0.001731,0.212350,-0.028103,-0.223460,-0.035469,0.113486,0.049002,1311,3.85,"Zoboi, Ibi || Pride",Pride,"Zoboi, Ibi",3.117934
1,hash_8466618ac1ae058828011a26e7b5e60f,-0.000184,0.664273,-0.024963,0.046442,-0.656345,-0.158792,-0.083407,121833,4.06,"Carr, Caleb || The Alienist (Dr. Laszlo Kreizl...","The Alienist (Dr. Laszlo Kreizler, #1)","Carr, Caleb",5.085769
2,hash_87ca483b2e90f93980ee25df09eacf99,0.005866,0.673671,-0.256699,0.306707,-0.057934,0.466045,0.164737,346200,4.30,"King, Stephen || 11/22/63",11/22/63,"King, Stephen",5.539328
3,hash_2032bf9577b6c4791aa1e00926f7aafd,0.006618,0.577222,-0.427983,0.464713,-0.430439,0.049712,-0.403589,2084695,4.54,"Rowling, J.K. || Harry Potter and the Goblet o...",Harry Potter and the Goblet of Fire (Harry Pot...,"Rowling, J.K.",6.319043
4,hash_f33c29ef8819e13f557be54a10d2e6c3,-0.005472,0.400177,-0.180042,0.322995,-0.159424,-0.331475,0.173746,121789,3.95,"Colfer, Eoin || The Arctic Incident (Artemis F...","The Arctic Incident (Artemis Fowl, #2)","Colfer, Eoin",5.085612


In [88]:
df2.to_csv('data/11-5 Plotting.csv')